In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses

import numpy as np
import cv2
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [ ]:
import os, glob, re, u_net, dataset_configs, data_io

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
str(datetime.datetime.now())

In [ ]:
df = pd.DataFrame({'age': range(0,10)})

In [ ]:
pd.cut(df['age'], list(range(0,10,3))+ [np.inf], right=False)

In [ ]:
np.logspace(-6, -1, 10)

In [ ]:
import sys, yaml
sys.path.append("/awlab/users/chsu/WorkSpace/tensorflow/segmentation/code")

In [ ]:
file = "/awlab/users/chsu/WorkSpace/tensorflow/segmentation/code/configs/ic_nuc_paper.yaml"
with open(file) as f:
    cfg = yaml.safe_load(f)

cfg['val_cfg']


In [ ]:
task = 'incucyte_nucleus'
test_size = 0.2
random_state = 423

data_cfg = dataset_configs.get_dataset_config(task)
data_cfg['match_pattern'] = '_[A-Z]4_4_00d00h'

x_train_fnames, x_val_fnames, y_train_fnames, y_val_fnames = \
    data_io.get_data_filenames(**data_cfg, test_size=test_size, random_state=random_state)

num_train_data = len(x_train_fnames)
num_val_data = len(x_val_fnames)

for k in data_cfg:
    print(k, ':', data_cfg[k])
print()
print("Number of training samples: {}".format(num_train_data))
print("Number of validation samples: {}".format(num_val_data))

In [ ]:
for f in x_train_fnames + x_val_fnames:
    print(os.path.basename(f))

In [ ]:
1048/16

In [ ]:
file_dir = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/images'

filter_pattern = '_A(?!2)\d+_1_00d00h'
fnames = get_filenames(file_dir, '*.png', filter_pattern)
print(len(fnames))

for f in fnames:
    print(os.path.basename(f))


In [ ]:
img = cv2.imread(r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/p2017017086_ki67_merge/images/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67_G9_0000.png')
img.max()

In [ ]:
conv_1x1 = layers.Conv2D(3, (1, 1), activation='softmax')

x = tf.random.normal([1, 3, 3, 10])
y_pred = conv_1x1(x)
y_true = tf.random.uniform([1, 3, 3], 0, 3, dtype=tf.int32)
y_true = tf.one_hot(y_true, 3)

print(y_pred)
print(y_true)

In [ ]:
# loss = losses.sparse_categorical_crossentropy(y_true, y_pred)
loss = losses.categorical_crossentropy(y_true, y_pred)
print(loss)

In [ ]:
L = np.zeros_like(loss)

for i in range(L.shape[1]):
    for j in range(L.shape[2]):
#         c = tf.squeeze(y_true[0,i,j])
        c = y_true[0,i,j,:].numpy() == 1.
#         print(c)
        L[0,i,j] = -tf.math.log(y_pred.numpy()[0,i,j,c])
print(L)

In [ ]:
L[:,:,1:]

# Tensorboard

In [ ]:
import tensorflow as tf
import datetime
import os

import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

In [ ]:
!tensorboard --logdir logs/fit

## Logging images

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [ ]:
logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

In [ ]:
!tensorboard --logdir logs/image